In [17]:
import re

class bag_rules:
    def __init__(self, your_bag_color, input_file='test.txt', verbose=False):
        self.your_bag_color = your_bag_color
        self.input_file = input_file
        self.verbose = verbose
        self.read_file()
        self.parse_file()

    def read_file(self):
        with open(self.input_file, 'r') as f:
            self.rules = f.read().splitlines()
    
    def parse_file(self):
        bags = {}
        for rule in self.rules:
            results = [x.group().strip() for x in re.finditer('(^.+?(?=bag))|(?=(\d))(.*?)(?=bag)', rule)]
            main_bag = results.pop(0)
            if main_bag not in bags.keys():
                bags[main_bag] = {}
            for result in results:
                a = [x.group().strip() for x in re.finditer('(^[0-9]+)|([a-z ]+)', result)]
                a[0] = int(a[0])
                bags[main_bag][a[1]] = a[0]
        self.bags = bags
    
    def check_contents(self, dictionary):
        return 1 if self.your_bag_color in dictionary.keys() else 0
    
    def check_sub_bags(self, d):        
        if  self.valid == 1:#or self.iterator >= len(self.bags.keys()):
            self.done = True
            return 
        
        for key in d.keys():
            if self.done: continue
            self.iterator += 1
            self.valid = max(self.check_contents(self.bags[key]), self.valid)
            if self.verbose: print(f"\tKEY={key}; VALID={self.valid}; ITER={self.iterator}")
            if self.valid == 1:
                self.done = True
            else:
                self.check_sub_bags(self.bags[key])
            
    
    def count_bags(self):
        n_outer_bags = 0
        for key in self.bags.keys():
            self.valid = 0
            self.iterator = 0
            self.done = False
            
            if key == self.your_bag_color: continue
            self.valid = self.check_contents(self.bags[key])
            
            if self.verbose: print(f"KEY={key}; VALID={self.valid};")
            while not self.done:
                self.check_sub_bags(self.bags[key])
                self.done=True
            n_outer_bags += self.valid
        return n_outer_bags
    
    def count_bags_part_2(self):
        self.n_inner_bags = 0
        for key in self.bags[self.your_bag_color]:
            self.done = False
            
            self.valid = self.check_contents(self.bags[key])
            
            if self.verbose: print(f"KEY={key}; VALID={self.valid};")
            while not self.done:
                self.check_sub_bags(self.bags[key])
                self.done=True
            n_outer_bags += self.valid
        return n_outer_bags

In [21]:
b = bag_rules('shiny gold')
print(b.rules)
print()
print(b.bags)
len(b.bags.keys())
b.count_bags()

['light red bags contain 1 bright white bag, 2 muted yellow bags.', 'dark orange bags contain 3 bright white bags, 4 muted yellow bags.', 'bright white bags contain 1 shiny gold bag.', 'muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.', 'shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.', 'dark olive bags contain 3 faded blue bags, 4 dotted black bags.', 'vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.', 'faded blue bags contain no other bags.', 'dotted black bags contain no other bags.']

{'light red': {'bright white': 1, 'muted yellow': 2}, 'dark orange': {'bright white': 3, 'muted yellow': 4}, 'bright white': {'shiny gold': 1}, 'muted yellow': {'shiny gold': 2, 'faded blue': 9}, 'shiny gold': {'dark olive': 1, 'vibrant plum': 2}, 'dark olive': {'faded blue': 3, 'dotted black': 4}, 'vibrant plum': {'faded blue': 5, 'dotted black': 6}, 'faded blue': {}, 'dotted black': {}}


4

In [22]:
b = bag_rules('shiny gold', input_file = 'test_02.txt')
print(b.rules)
print()
print(b.bags['shiny gold'])
len(b.bags.keys())
b.count_bags()

['shiny gold bags contain 2 dark red bags.', 'dark red bags contain 2 dark orange bags.', 'dark orange bags contain 2 dark yellow bags.', 'dark yellow bags contain 2 dark green bags.', 'dark green bags contain 2 dark blue bags.', 'dark blue bags contain 2 dark violet bags.', 'dark violet bags contain no other bags.']

{'dark red': 2}


0

In [59]:

for key, values in b.bags['shiny gold'].items():
    print(key, values)

dark red 2


NameError: name 'bags' is not defined

In [ ]:
# two components 
# first find from the beginning of the string to the first instance of bag
# second find between each digit and the following bag
pattern = '(^.+?(?=bag))|(?=(\d))(.*?)(?=bag)' 

In [63]:
!pip install anytree

  Using cached https://artifactory.service.bo1.csnzoo.com/artifactory/api/pypi/pypi-mirror/packages/a8/65/be23d8c3ecd68d40541d49812cd94ed0f3ee37eb88669ca15df0e43daed1/anytree-2.8.0-py2.py3-none-any.whl
You are using pip version 9.0.3, however version 20.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [87]:
from anytree import Node, RenderTree, find
root = Node("root")

for key, value in b.bags.items():
    n = Node(key, parent=root)
    for k,v in b.bags[key].items():
        for i in range(v):
            nn = Node(k, parent=n)

for pre, fill, node in RenderTree(root):
    print("%s%s" % (pre, node.name))


root
├── shiny gold
│   ├── dark red
│   └── dark red
├── dark red
│   ├── dark orange
│   └── dark orange
├── dark orange
│   ├── dark yellow
│   └── dark yellow
├── dark yellow
│   ├── dark green
│   └── dark green
├── dark green
│   ├── dark blue
│   └── dark blue
├── dark blue
│   ├── dark violet
│   └── dark violet
└── dark violet


In [82]:
for n in root.children:
    print(n.name)

shiny gold
dark red
dark orange
dark yellow
dark green
dark blue
dark violet


In [108]:
def count_children(n, bags):
    if len(n.children) > 0:
        for child in n.children:
            bags += 1
            ref_node =  find(root, lambda node: node.name == child.name, maxlevel=2)
            bags = count_children(ref_node, bags)
    return bags

bags = 0
n = find(root, lambda node: node.name == 'shiny gold', maxlevel=2)
count_children(n, bags)


Node('/root/dark red')
Node('/root/dark orange')
Node('/root/dark yellow')
Node('/root/dark green')
Node('/root/dark blue')
Node('/root/dark violet')
Node('/root/dark violet')
Node('/root/dark blue')
Node('/root/dark violet')
Node('/root/dark violet')
Node('/root/dark green')
Node('/root/dark blue')
Node('/root/dark violet')
Node('/root/dark violet')
Node('/root/dark blue')
Node('/root/dark violet')
Node('/root/dark violet')
Node('/root/dark yellow')
Node('/root/dark green')
Node('/root/dark blue')
Node('/root/dark violet')
Node('/root/dark violet')
Node('/root/dark blue')
Node('/root/dark violet')
Node('/root/dark violet')
Node('/root/dark green')
Node('/root/dark blue')
Node('/root/dark violet')
Node('/root/dark violet')
Node('/root/dark blue')
Node('/root/dark violet')
Node('/root/dark violet')
Node('/root/dark orange')
Node('/root/dark yellow')
Node('/root/dark green')
Node('/root/dark blue')
Node('/root/dark violet')
Node('/root/dark violet')
Node('/root/dark blue')
Node('/root/da

126

In [101]:
bags = 1


In [73]:
node_dict

{'shiny gold': Node('/shiny gold'),
 'dark red': Node('/dark red'),
 'dark orange': Node('/dark orange'),
 'dark yellow': Node('/dark yellow'),
 'dark green': Node('/dark green'),
 'dark blue': Node('/dark blue'),
 'dark violet': Node('/dark violet')}

In [57]:
n_bags = 0
def count_sub_bags(color, d, n_bags=0):
    while d[color] != {}:
        for key, value in d[color].items():
            
        
        
current_color = 'shiny gold'

:
    for key, value in start.items():
        holding_bags = value
        print(b.bags[key], holding_bags)

IndentationError: expected an indented block (<ipython-input-57-92ea353ebe01>, line 4)

In [61]:
chained_bags = 1

n_bags = 1

root = 'shiny gold'
for key, value in b.bags[root].items():
    n_bags += value
    done = False
    while done = False
        n_bags += value*get_sub_bags(b.bags[key])
        
        print(key, value)
        print(b.bags[key])

dark red 2
{'dark orange': 2}


In [62]:
n_bags

3